# CNN을 활용한 MNIST 모델

In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


# CNN(Convolution Neural Network)란?
CNN이란 컨볼루션 뉴럴 네트워크로, 컨볼루션 레이어와 풀링 레이어로 구성된 Neural Network이다.
영상, 이미지등의 인식률이 높아 주로 Image & Video Recognition에 사용되며 구조는 다음 그림과 같다.
![images](./images/CNN.png)

# Convolution Layer(전처리 단계)
Convolution Layer는 일반적으로 전처리 단계를 의미한다. 다양한 필터와 Convolution Kernel등을 통해서 사진 인식에 중요한 Feature만을 요약한 사진이라고 할 수 있는 Feature Map을 생성한다.
![images2](./images/CNN2.png)
아래 그림은 Input으로 들어온 사진에다가 필터를 이용한 행렬곱을 통해 십자가 포인트를 찾는 과정을 보여준다.
![images3](./images/CNN3.png)  

위 십자가 포인트를 바탕으로 생성된 Feature Map은 다음과 같다. 여기에 ReLU연산을 함으로써, 0보다 작은 값들을 0으로 만들어준뒤, Feature Map을 보다 효율적으로 학습하기 위해 사이즈를 줄여가며 요약하는 Pooling 과정을 거쳐야 한다.
![images4](./images/CNN4.png)

# Pooling Layer(전처리 단계)
Pooling Layer는 Convolution Layer의 결과로 나온 Feature Map을 요약하고 압축하는 과정을 시행하는 Layer이다. 일반적으로 사용되는 Pooling은 Max Pooling이다. Max Pooling은 특정 구역의 최대값을 추출하여 정보를 요약한다.
![images5](./images/CNN5.png)

위의 그림은 2x2 Kernel을 사용하는 Max Pooling이다. Stride는 2로 설정되어 있는데, 이 때 Stride는 한번에 움직일 칸수를 의미한다.

# 전처리 단계 개괄도
![images6](./images/CNN6.png)

CNN은 Convolution Layer에서 Input Image에 특정한 Filter를 곱합으로써, Feature Map을 구하고, 이 Feature Map에서 Pooling을 통해 요약된 정보를 다음 레이어의 Input으로 전달하는 방식이다.

# 분류 단계
Convolution Layer와 Pooling Layer를 거쳐서 최종적으로 요약된 데이터들을 Fully Conntected Layer에서 사용하여 이미지를 분석하고, 분류한다.
![images7](./images/CNN7.png)

## Neural Network Contruction

In [7]:
X= tf.placeholder(tf.float32, [None,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

기존 모델에서는 입력 값을 28x28인 784짜리 일차원으로 구성하였으나, CNN 모델을 활용하기 위해서 2차원 평면과 특성치의 형태를 가지는 구조로 설정한다.

In [9]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
#L1 = tf.nn.dropout(L1,keep_prob)

W1 [3 3 1 32] -> [3 3] : Kernel Size, 1: 입력값 X의 특성수, 32 : Filter의 개수  
L1 Conv shape = (None, 28, 28, 32)  
L1 Pool Shape = (None, 14, 14 , 32)
### Padding이란?
padding='SAME' 옵션은 pooling과 convolution시에 해상도가 작아지는 것을 방지하기 위해서, 다음과 같이 테두리 하나가 더 있다고 가정하고 움직이게끔 하는 옵션이다. 
![images8](http://xrds.acm.org/blog/wp-content/uploads/2016/06/Figure_3.png)

In [10]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
#L2 = tf.nn.dropout(L2,keep_prob)

L2 Conv shape = (None, 14, 14, 64)  
L2 Pool Shape = (None, 7, 7 , 64)  
W2의 [3, 3, 32, 64] 에서 32는 L1에서 출력된 W1의 마지막 차원, 즉 Filter의 개수 32이다.

In [11]:
W3 = tf.Variable(tf.random_normal([7*7*64,256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3,W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3,keep_prob)

FC레이어 : 입력값 7x7x64 -> 출력값은 256  
Full Connect를 위해 직전의 Pool Size인 (None, 7, 7, 64)를 참고하여 차원을 줄여준다.

In [13]:
W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
model = tf.matmul(L3,W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
#optimizer = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)

# Neural Network 학습

In [18]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs =  batch_xs.reshape(-1,28,28,1)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        total_cost += cost_val
    
    print('Epoch :', '%04d' % (epoch+1), 'Avg.cost =', '{:.3f}'.format(total_cost / total_batch))
    
print('Optimization Complete!')

('Epoch :', '0001', 'Avg.cost =', '0.364')
('Epoch :', '0002', 'Avg.cost =', '0.113')
('Epoch :', '0003', 'Avg.cost =', '0.083')
('Epoch :', '0004', 'Avg.cost =', '0.063')
('Epoch :', '0005', 'Avg.cost =', '0.052')
('Epoch :', '0006', 'Avg.cost =', '0.045')
('Epoch :', '0007', 'Avg.cost =', '0.038')
('Epoch :', '0008', 'Avg.cost =', '0.034')
('Epoch :', '0009', 'Avg.cost =', '0.029')
('Epoch :', '0010', 'Avg.cost =', '0.026')
('Epoch :', '0011', 'Avg.cost =', '0.022')
('Epoch :', '0012', 'Avg.cost =', '0.020')
('Epoch :', '0013', 'Avg.cost =', '0.018')
('Epoch :', '0014', 'Avg.cost =', '0.016')
('Epoch :', '0015', 'Avg.cost =', '0.016')
Optimization Complete!


# 결과 확인

In [21]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
                                   Y: mnist.test.labels,
                                   keep_prob: 1}))

('Accuracy:', 0.9895)
